In [1]:
import pandas as pd
import numpy as np
import torch



def read_file(fname: str, correct_labels=False) -> pd.DataFrame:
    """Reads a filename, return df with text and labels."""

    df = pd.read_table(fname, sep="\t", header=None, names="text,labels,role".split(","))
    if correct_labels:
        offensive_ids = df.labels != "Acceptable speech"
        df.labels[offensive_ids] = 1
        df.labels[~offensive_ids] = 0

    df = df.drop(columns=["role"])
    return df

en_test, en_train = "../data/merged-en.test.tsv" , "../data/merged-en.train.tsv"
hr_test, hr_train = "../data/merged-hr.test.tsv" , "../data/merged-hr.train.tsv"
sl_test, sl_train = "../data/merged-sl.test.tsv",  "../data/merged-sl.train.tsv"


In [2]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import wandb


model_args = ClassificationArgs()
model_args.use_early_stopping = True
model_args.early_stopping_delta = 0.01
model_args.early_stopping_metric = "mcc"
model_args.early_stopping_metric_minimize = False
model_args.early_stopping_patience = 3
model_args.evaluate_during_training_steps = 1000

model_args.evaluate_during_training = True
model_args.manual_seed = 4
model_args.use_multiprocessing = True
model_args.eval_batch_size = 8
model_args.labels_list = [0, 1]
model_args.wandb_project = "task2"
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True


sweep_config = {
    "method": "bayes",  # grid, random
    "metric": {"name": "mcc", "goal": "maximize"},
    "parameters": {
        "num_train_epochs": {"values": [5, 10, 20]},
        "learning_rate": {"values": [0.00002927]},
        "train_batch_size": {"values": [40,80,160]}
    },
}

sweep_id = wandb.sweep(sweep_config, project="task2-cont")

train_df = read_file(hr_train, correct_labels=True)
eval_df = read_file(hr_test, correct_labels=True)

def train():
    # Initialize a new wandb run
    wandb.init()

    # Create a TransformerModel
    model = ClassificationModel(
        "electra",
        "classla/bcms-bertic",
        use_cuda=True,
        args=model_args,
        sweep_config=wandb.config,
    )

    # Train the model
    model.train_model(train_df, eval_df=eval_df)

    # Evaluate the model
    model.eval_model(eval_df)

    # Sync wandb
    wandb.join()

wandb.agent(sweep_id, train)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: qvhsksia
Sweep URL: https://wandb.ai/5roop/task2-cont/sweeps/qvhsksia


wandb: Agent Starting Run: 9g63ch48 with config:
wandb: 	learning_rate: 2.927e-05
wandb: 	num_train_epochs: 10
wandb: 	train_batch_size: 40
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 5roop (use `wandb login --relogin` to force relogin)


Some weights of the model checkpoint at classla/bcms-bertic were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at classla/bcms-bertic and are newly initialized: ['pooler.dense.bias', 'classifier.bias', '

/home/peterr/anaconda3/lib/python3.8/site-packages/simpletransformers/classification/classification_model.py:941: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  torch.nn.utils.clip_grad_norm_(


...

Training loss,0.00067
lr,0.0
global_step,2220
_runtime,395
_timestamp,1629276441
_step,58
tp,1134
tn,633
fp,196
fn,157
mcc,0.6479


Training loss,█▅▆▅▅▅▅▅▄▃▃▄▃▁▃▂▂▃▁▃▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁
lr,▄▆███▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁
global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
tp,▄▁█▁▃▃▅▅▆▄▄▄
tn,▅█▁█▆▅▄▄▄▅▅▅
fp,▄▁█▁▃▄▅▅▅▄▄▄
fn,▅█▁█▆▆▄▄▃▅▅▅
mcc,▁█▂▇▅▃▅▅▄▆▇▇


wandb: Agent Starting Run: vzmn4cfh with config:
wandb: 	learning_rate: 2.927e-05
wandb: 	num_train_epochs: 5
wandb: 	train_batch_size: 80
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Some weights of the model checkpoint at classla/bcms-bertic were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at classla/bcms-bertic and are newly initialized: ['pooler.dense.bias', 'classifier.bias', '

wandb: Ctrl + C detected. Stopping sweep.


In [21]:
train_df = read_file(hr_train, correct_labels=True)
eval_df = read_file(hr_test, correct_labels=True)

eval_df

,text,labels
0,"Ovi ne idu nigdje za boljim životom, pošto im ...",1
1,Na zalost sve moguce,0
2,Ocemu se tu radi ne kuzim,0
3,Tako je kad vi pisete istoriju i pola miliona ...,1
4,Meni se ovo jako svida to smo htjeli nasi hrva...,1
...,...,...
2115,Nekome trebaju naočale. Ovo je šezdesetak ljud...,0
2116,Slusate li vi uopce sta oni gove?,0
2117,Jadno..,1
2118,"pa mi stvarno nismo normalni kao narod, zasto ...",1
